## ДЗ 1. Класс kmeans

Дано:
- дан класс K_Means кластеризации данных
Требуется:
- изменить способ задания начальных положений центров кластеров: разбросать случайным образом между(!) точек заданного обучающего множества dataset (определив разброс точек по всем координатам)
- изменить способ задания параметра tolerance: значение параметра должно равняться 1/10000 от минимального диапазона изменения признаков объектов (например, если один признак изменятся в диапазоне [100,200], а другой - [0.1,10], то tolerance = (10-0.1)/10000)
- дописать метод get_dist: пользователь должен иметь возможность использовать одну из 4 метрик при подсчете расстояний между объектами (евклидово расстояние, квадрат евклидова расстояния, расстояние городских кварталов, расстояние Чебышёва)
- написать метод get_predict: методом можно воспользоваться только после обучения модели (fitted == True), метод принимает двумерный массив объектов (могут быть точки не принадлежащие исходному обучающему множеству dataset), метод возвращает одномерный массив индексов соответствующих кластеров (по близости до центров кластеров)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class K_means():

    def __init__(self, dataset, n_clusters=3):
        self.dataset = dataset
        self.n_clusters = n_clusters
        self.max_n_iter = 100
        self.tolerance = 0.01
        self.fitted = False
        self.labels = np.array([], dtype='i')
        self.centroids = np.array([self.dataset[k] for k in
                                  range(self.n_clusters)], dtype='f')

    def get_dist(self, list1, list2):
        return np.sqrt(sum([(i-j)**2 for i,j in zip(list1,list2)]))

    def distribute_data(self):
        self.labels = np.array([], dtype='i')
        for data in self.dataset:
            dist = np.array([self.get_dist(data,center)
                            for center in self.centroids])
            self.labels = np.append(self.labels, dist.argmin())

    def recalculate_centroids(self):
        for i in range(self.n_clusters):
            num = 0
            temp = np.empty(self.dataset[0].shape)
            for k,label in enumerate(self.labels):
                if label == i:
                    temp += self.dataset[k]
                    num += 1
            self.centroids[i] = temp / num

    def fit(self):
        iter = 1
        while iter < self.max_n_iter:
            prev_centroids = self.centroids.copy()
            self.distribute_data()
            self.recalculate_centroids()
            if max([self.get_dist(i,j) for i,j in zip(prev_centroids,self.centroids)]) \
            < self.tolerance:
                break
            iter += 1
        self.fitted = True

    '''
    def predict(self, list2d):
        return list1d -> [0, 1, 1, 2, ...]
    '''




kmeans = K_means(points)
print(kmeans.centroids)
kmeans.get_dist([1,0,1,2],[9,4,1,0])
#print(kmeans.labels)
kmeans.distribute_data()
#print(kmeans.labels)
kmeans.recalculate_centroids()
print(kmeans.centroids)

kmeans.fit()

fig = plt.figure(figsize=(width/60,height/60))
plt.scatter(points[:,0],points[:,1],c=kmeans.labels)
plt.scatter(kmeans.centroids[:,0],kmeans.centroids[:,1],c='black',marker='+')
plt.show()